In [56]:
import importlib 

import ball
from ball import Ball
importlib.reload(ball)

from scipy.stats import boltzmann

import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import HTML

In [57]:
num_frames = 100

a = int(input("Enter rows:"))
b = int(input("Enter columns:"))

n = [a, b]
nx = n[0]
ny = n[1]

balls = []
velocity_multiplier = float(input("Enter the velocity multiplyer:"))
first_ball = True

In [58]:
# set up the figure and axis
plt.ioff() # stop automatic inline display
fig, ax = plt.subplots()
ax.set_xlim(0, n[0])
ax.set_ylim(0, n[1])
ax.set_aspect('equal')
# plt.gca().invert_yaxis() # origin top left

# obtain positions for nxn grid
for i in range(nx):
    for j in range(ny):
        # create a dot at an initial dummy position
        circle, = ax.plot([], [], marker="o", markersize=10)
        ball = Ball([abs(i + 0.5), abs(j + 0.5)], circle)
        ball.velocity_y = 0
        balls.append(ball)

In [ ]:
# animation update function
def update(frame):
    for ball in balls:
        # --- update position ---

        # if first_ball:
        #     ball_color = 'ro'

        circle = ball.circle
        x = ball.position[0]
        y = ball.position[1]
        velocity_x =  ball.velocity_x * velocity_multiplier
        velocity_y = ball.velocity_y * velocity_multiplier
        direction_x = ball.direction_x
        direction_y = ball.direction_y 
        colliding = ball.colliding

        if x >= (n[0]):
            direction_x = -1
        elif x <= 0:
            direction_x = 1

        if y >= (n[1]):
            direction_y = -1
        elif y <= 0:
            direction_y = 1

        # finalize positions for that frame
        x = [(x + (velocity_x * direction_x))]
        y = [(y + (velocity_y * direction_y))]

        circle.set_data(x, y)

        # --- collision ---
        cell = [int(x[0]), int(y[0])] # each cell is 1 x 1
        cell_x = cell[0]
        cell_y = cell[1]
        adjacent_cells = []
        adjacent_cells_balls = []

        # - corners -
        if cell_x == 0 and cell_y == 0:                 # LEFT top
            adjacent_cells = [[cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_x == nx and cell_y == 0:              # RIGHT top
            adjacent_cells = [[cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y]]
        elif cell_x == nx and cell_y == ny:             # RIGHT top
            adjacent_cells = [[cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1],
                              [cell_x, cell_y - 1]]
        elif cell_x == 0 and cell_y == ny:              # LEFT bottom
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y]]

        # - edges -
        elif cell_x == 0:                               # LEFT
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_y == 0:                               # TOP
            adjacent_cells = [[cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y]]
        elif cell_x == nx:                              # RIGHT
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x - 1, cell_y - 1],
                              [cell_x - 1, cell_y],
                              [cell_x - 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_y == ny:                              # BOTTOM
            adjacent_cells = [[cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1],
                              [cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y]]

        # - center -
        else:
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1]]

        for next_ball in balls:
            next_x = next_ball.position[0]
            next_y = next_ball.position[1]
        
            next_cell = [int(next_x), int(next_y)]

            if next_cell in adjacent_cells:
                if next_ball != ball:
                    adjacent_cells_balls.append(next_ball)
        
        for adj_ball in adjacent_cells_balls:
            x2 = adj_ball.position[0]
            y2 = adj_ball.position[1]
            distance = math.sqrt(((x2 - x[0]) ** 2) + ((y2 - y[0]) ** 2))
            print("ball #:", balls.index(ball), "\n")
            # print("dist:", distance, "\n")
            print("x1:", x[0])
            print("x2:", x2, "\n")

            if distance <= 0.1 and not colliding:
                print("collision detected")
                colliding = True
            elif distance > 0.1 and colliding:
                colliding = False

        # update inherent ball position
        ball.circle = circle
        ball.position = [x[0], y[0]]
        ball.direction_x = direction_x
        ball.direction_y = direction_y
        ball.colliding = colliding

In [60]:
# animation
ani_obj = animation.FuncAnimation(fig, update, frames=num_frames, interval=50)
HTML(ani_obj.to_jshtml()) # render inline

ball #: 0 

x1: 0.545613674719518
x2: 1.5 

ball #: 1 

x1: 1.494988514748692
x2: 0.545613674719518 

ball #: 0 

x1: 0.5912273494390361
x2: 1.494988514748692 

ball #: 1 

x1: 1.4899770294973842
x2: 0.5912273494390361 

ball #: 0 

x1: 0.6368410241585541
x2: 1.4899770294973842 

ball #: 1 

x1: 1.4849655442460763
x2: 0.6368410241585541 

ball #: 0 

x1: 0.6824546988780722
x2: 1.4849655442460763 

ball #: 1 

x1: 1.4799540589947684
x2: 0.6824546988780722 

ball #: 0 

x1: 0.7280683735975902
x2: 1.4799540589947684 

ball #: 1 

x1: 1.4749425737434605
x2: 0.7280683735975902 

ball #: 0 

x1: 0.7736820483171083
x2: 1.4749425737434605 

ball #: 1 

x1: 1.4699310884921526
x2: 0.7736820483171083 

ball #: 0 

x1: 0.8192957230366263
x2: 1.4699310884921526 

ball #: 1 

x1: 1.4649196032408447
x2: 0.8192957230366263 

ball #: 0 

x1: 0.8649093977561444
x2: 1.4649196032408447 

ball #: 1 

x1: 1.4599081179895368
x2: 0.8649093977561444 

ball #: 0 

x1: 0.9105230724756624
x2: 1.4599081179895368 
